In [ ]:
%pip install mediapipe opencv-python pyngrok
%pip install pyautogui 

In [ ]:
import cv2
import mediapipe as mp
import pyautogui  # For mouse control and scrolling

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Initialize camera
cap = cv2.VideoCapture(0)

# Numeric value for scrolling
numeric_value = 50

# Flags
click_flag = False
scroll_flag = False
mouse_position = (0, 0)  # Mouse position tracker

# Define buttons for the interface
buttons = [
    {"label": "Button 1", "x1": 50, "y1": 50, "x2": 200, "y2": 100, "color": (200, 200, 200), "hover": False, "clicked": False},
    {"label": "Button 2", "x1": 50, "y1": 150, "x2": 200, "y2": 200, "color": (200, 200, 200), "hover": False, "clicked": False},
]

# Mouse callback for hover and click
def mouse_callback(event, x, y, flags, param):
    global mouse_position, buttons
    mouse_position = (x, y)

    # Handle mouse click
    if event == cv2.EVENT_LBUTTONDOWN:
        for button in buttons:
            if button["x1"] <= x <= button["x2"] and button["y1"] <= y <= button["y2"]:
                if not button["clicked"]:  # Prevent double clicks
                    button["clicked"] = True
                    print(f"{button['label']} clicked")

# Set up the mouse callback
cv2.namedWindow("Hand Gesture Recognition with Interface")
cv2.setMouseCallback("Hand Gesture Recognition with Interface", mouse_callback)

# MediaPipe Hands instance
with mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7, min_tracking_confidence=0.7) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame.")
            break

        # Flip frame for mirror effect
        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Hand detection
        results = hands.process(rgb_frame)

        # Get screen dimensions
        screen_width, screen_height = pyautogui.size()

        # Draw buttons and handle hover/click
        for button in buttons:
            # Determine hover state
            if button["x1"] <= mouse_position[0] <= button["x2"] and button["y1"] <= mouse_position[1] <= button["y2"]:
                button["hover"] = True
                color = (0, 255, 255)  # Yellow hover color
            else:
                button["hover"] = False
                color = button["color"]

            # Change color if clicked
            if button["clicked"]:
                color = (0, 255, 0)  # Green clicked color

            # Draw the button
            cv2.rectangle(frame, (button["x1"], button["y1"]), (button["x2"], button["y2"]), color, -1)
            cv2.putText(frame, button["label"], (button["x1"] + 10, button["y1"] + 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

        # Display numeric value
        cv2.putText(frame, f"Value: {numeric_value}", (300, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (139, 0, 139), 2)

        # Hand landmarks processing
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Get finger landmarks
                thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
                thumb_ip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP]
                index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
                ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
                pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]

                # Move the cursor using index fingertip
                cursor_x = int(index_tip.x * screen_width)
                cursor_y = int(index_tip.y * screen_height)
                pyautogui.moveTo(cursor_x, cursor_y)
                cv2.putText(frame, "Cursor Moving", (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)  # Beyaz

                # Gesture-based clicking
                if (index_tip.y < middle_tip.y and
                        ring_tip.y > middle_tip.y and pinky_tip.y > middle_tip.y):
                    if not click_flag:
                        click_flag = True
                        pyautogui.click()  # Perform click globally
                        cv2.putText(frame, "Click Detected", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                        # Check if the cursor is over a button
                        for button in buttons:
                            if button["hover"]:
                                if not button["clicked"]:  # Prevent double clicks
                                    button["clicked"] = True
                                    print(f"{button['label']} clicked")
                else:
                    click_flag = False

                # Scroll down gesture (thumb down)
                thumb_down = thumb_tip.y > thumb_ip.y + 0.05
                if thumb_down and abs(thumb_tip.x - thumb_ip.x) < 0.1 and not scroll_flag:
                    numeric_value = max(0, numeric_value - 1)
                    pyautogui.scroll(-30)
                    scroll_flag = True
                    cv2.putText(frame, "Scroll Down", (300, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                elif not thumb_down:
                    scroll_flag = False

                # Scroll up gesture (thumbs up)
                thumb_up = (thumb_tip.y < thumb_ip.y - 0.05 and
                            all(finger.y > thumb_tip.y + 0.05 for finger in [index_tip, middle_tip, ring_tip, pinky_tip]))
                if thumb_up and abs(thumb_tip.x - thumb_ip.x) < 0.1 and not scroll_flag:
                    numeric_value = min(100, numeric_value + 1)
                    pyautogui.scroll(30)
                    scroll_flag = True
                    cv2.putText(frame, "Scroll Up", (300, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                elif not thumb_up:
                    scroll_flag = False

        # Display the frame
        cv2.imshow("Hand Gesture Recognition with Interface", frame)

        # Exit loop on 'q' press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release resources
cap.release()
cv2.destroyAllWindows()